In [246]:
import requests

try:
    print("Loading shared code from GitHub...")
    url = 'https://raw.githubusercontent.com/peterdresslar/Dresslar_CAS522/main/src/shared.py'
    response = requests.get(url)
    response.raise_for_status()
    exec(response.text)
    print("Successfully loaded shared code.")
except requests.exceptions.RequestException as e:
    print(f"Failed to load code from GitHub: {e}")
    print("\nPlease ensure you have an internet connection, or place the shared.py file in a 'src' directory next to this notebook.")

Loading shared code from GitHub...
Successfully loaded shared code.


## Dresslar CAS522 M6 Notebook


In [247]:
import os
import sys
import numpy as np
import itertools
import textwrap
from collections import OrderedDict

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.shared import int_less_than_21_to_word  # adjust if needed


Consider the dynamics of the following Boolean network (Fig. 1.)

<br />
<img src="../public/Assignment Module 6-1.png" width="400px"/> <br />
<span text-size="small"><em>Figure 1</em><span>
<br />


### Question 1
> Determine the global dynamics of the network by finding all cycles and fix-point attractors. Determine the sizes of all basins of attraction. Perform the analysis without using any numerical code.

To begin, we should describe this diagram in the same manner as (Gros p.80): this is a boolean network with $N = 4$ sites and connectivities $K_i = 2$. The diagram as it is contains "network linkage and coupling functions." We should be able to use these to diagram the network dynamics.

Figure 2 contains the (hand-drawn) "complete network dynamics" (Gros p.80), along with a complete state table on the right side of the image.

<br />
<img src="../public/M6_dynamics.jpg" width="500px"/> <br />
<span text-size="small"><em>Figure 2</em><span>
<br /> <br />

From this diagram that we can see that we have two fixpoint attractors, `0000` and `1111`, each of which supplies a basin of attraction for a two transient states (respectively: `0001`, `0100` and `0111`, `1101`). We also have a length 2 cyclic attractor `0101 <--> 1010`: interestingly, while `1010` captures only the output of `0101`, the latter is the entrypoint of the cyclic basin for all the remaining transient states.


### Question 2
> Then double-check your results in python, and submit your notebook. 

Terrific! No more dry-erase zeroes for today.


In [248]:
# State, following Gros:
N = 4
K = 2
figure = 1

state = np.zeros(N, dtype=bool)

def f_sigma_1(state):
    return state[1] & state[3]

def f_sigma_2(state):
    return state[0] | state[2]

def f_sigma_3(state):
    return state[1] & state[3]

def f_sigma_4(state):
    return state[0] | state[2]

def update(state):
    sigma_0 = f_sigma_1(state)
    sigma_1 = f_sigma_2(state)
    sigma_2 = f_sigma_3(state)
    sigma_3 = f_sigma_4(state)
    state = np.array([sigma_0, sigma_1, sigma_2, sigma_3])
    return state

# okay, it is very easy to simply compute all 2^N states and their first updates,
# this should look just like the right side of my whiteboard.
print("start --> next")
for i in range(2**N):
    # many ways to do this but let's be explict by converting i to binary:
    i_bin = np.binary_repr(i, width=N)
    # now we can write i_bin to state:
    state = np.array([int(x) for x in i_bin])  # this is kind of cheating but works fine
    # and then we can simply print the state and its update:
    print(f"{state} --> {update(state)}")


start --> next
[0 0 0 0] --> [0 0 0 0]
[0 0 0 1] --> [0 0 0 0]
[0 0 1 0] --> [0 1 0 1]
[0 0 1 1] --> [0 1 0 1]
[0 1 0 0] --> [0 0 0 0]
[0 1 0 1] --> [1 0 1 0]
[0 1 1 0] --> [0 1 0 1]
[0 1 1 1] --> [1 1 1 1]
[1 0 0 0] --> [0 1 0 1]
[1 0 0 1] --> [0 1 0 1]
[1 0 1 0] --> [0 1 0 1]
[1 0 1 1] --> [0 1 0 1]
[1 1 0 0] --> [0 1 0 1]
[1 1 0 1] --> [1 1 1 1]
[1 1 1 0] --> [0 1 0 1]
[1 1 1 1] --> [1 1 1 1]


In [249]:
# But now we want to find all cycles and fix-point attractors. We are fortunate to have a useful constraint, here: no cycle
# can be longer than 2^N = 16 steps. Thus, brute force is easy. So, the following might be a useful overview, if not exactly what we want.

print("start --> sigma_1 --> sigma_2 --> ... --> sigma_2^N")
for i in range(2**N):
    i_bin = np.binary_repr(i, width=N)
    state = np.array([int(x) for x in i_bin])
    # and then we can piece together a single line of update outputs:
    line = f"{state}"
    for j in range(2**N):
        new_state = update(state)
        line += f" --> {new_state}"
        state = new_state
    print(line)

start --> sigma_1 --> sigma_2 --> ... --> sigma_2^N
[0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0]
[0 0 0 1] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0] --> [0 0 0 0]
[0 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0]
[0 0 1 1] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0] --> [0 1 0 1] --> [1 0 1 0]
[0 1 0 0] --

In [250]:
# .... We are going to need to do a lot better. Basically we need to track our states and search. What we can see from the view above is that
# the pattern we are searching for is when a state *repeats* in the sequence. Then, the sequence of states between repeats is our cycle. We might 
# track that sequence as an array, since our first and last states are important for us to understand our system dynamics.
# (commented prints can be uncommented to see the process)

def get_idx_of_np_array_in_list(np_array_to_check, list_np_arrays):
    # returns -1 if not found: otherwise returns index of first occurence
    #print(f"list_np_arrays: {len(list_np_arrays)}")
    for i, arr in enumerate(list_np_arrays):
        #print(f"testing {np_array_to_check} against {arr}")
        if np.array_equal(np_array_to_check, arr):
            #print(f"found at index {i}")
            return i
    return -1


print("start --> cycle: cycle length")
print("--------------------------------")
for i in range(2**N):
    i_bin = np.binary_repr(i, width=N)
    starting_state = np.array([int(x) for x in i_bin])
    state = starting_state
    path = [state]
    while True:
        new_state = update(state)
        # check j_vals to see if there is already an entry for new_state
        # note we are using numpy, so it is a bit more work to check if an array is in a list of arrays
        idx_if_present = get_idx_of_np_array_in_list(new_state, path)  # get index or -1
        if idx_if_present > -1: # we have seen this state before
            cycle_start = idx_if_present
            cycle = path[idx_if_present:] 
            print(f"{i_bin} --> {cycle}: cycle length {len(cycle)}")
            break
        state = new_state
        path.append(new_state)



start --> cycle: cycle length
--------------------------------
0000 --> [array([0, 0, 0, 0])]: cycle length 1
0001 --> [array([0, 0, 0, 0])]: cycle length 1
0010 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
0011 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
0100 --> [array([0, 0, 0, 0])]: cycle length 1
0101 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
0110 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
0111 --> [array([1, 1, 1, 1])]: cycle length 1
1000 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
1001 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
1010 --> [array([1, 0, 1, 0]), array([0, 1, 0, 1])]: cycle length 2
1011 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
1100 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
1101 --> [array([1, 1, 1, 1])]: cycle length 1
1110 --> [array([0, 1, 0, 1]), array([1, 0, 1, 0])]: cycle length 2
1111 --> [array([1, 1,

This is a nice "first" attempt at getting the information we want about system dynamics, but a few tweaks will allow us to clean up the data. We especially need to "de-duplicate" the cycles and we want to be able to count and size them. 

The following code block is intended to deliver a "textbook" answer (again, Gros (p. 80)).

In [251]:
def cycle_to_string(cycle):
    """Inputs:
    [array([0, 1, 0, 1]), array([1, 0, 1, 0])]
    Outputs:
    ['0101', '1010']
    """
    out = []
    for c in cycle:
        out.append("".join([str(int(not x)) for x in c]))
    return out

def test_if_cycle_identical(cycle1, cycle2):
    """
    Inputs:
    ['0101', '1010', foo, bar]
    ['1010', foo, bar,'0101']
    Outputs:
    True
    """
    identical = False
    if len(cycle1) != len(cycle2):
        return identical
    for i, cyc1 in enumerate(cycle1):
        for j, cyc2 in enumerate(cycle2):
            # find the element i in cycle2
            if cyc1 == cyc2:
                # create two new lists with i, j as the starting indicies
                cyc1_new = cycle1[i:] + cycle1[:i]
                cyc2_new = cycle2[j:] + cycle2[:j]
                # print(f"testing {cyc1_new} against {cyc2_new}")
                if cyc1_new == cyc2_new:
                    identical = True
                    return identical

    return identical

# c1 = ['0101', '1010', 'foo', 'bar']
# c2 = ['1010', 'foo', 'bar', '0101']
# c3 = ['1010', 'foo', 'bar', '0101']
# c4 = ['1010', 'bar', 'foo', '0101'] 
# print(test_if_cycle_identical(c1, c2))
# # True
# print(test_if_cycle_identical(c3, c4))
# # False


def remove_duplicates_and_sort(cycles_list):
    """
    Sort is longest to shortest and then alpha
    Inputs:
    [['0000'], ['0101', '1010'], ['0101', '1010']]
    Outputs:
    [['0101', '1010'], ['0000']]
    """

    # print(type(cycles_list), type(cycles_list[0]), type(cycles_list[0][0]))
    # <class 'list'> <class 'list'> <class 'str'>

    # First we do the easy part:
    # we need to remove all exact duplicates
    # https://stackoverflow.com/a/2213973

    k = sorted(cycles_list, key=lambda x: x[0])  # sorted to return new list
    cl = list(k for k,_ in itertools.groupby(k))
    # print(cl) debugging was a pain!

    # now with our remaining list, we need to check each cycle against every other cycle
    dups = set() # unique vals!
    for m in range(len(cl)):
        # if we already marked this cycle as a duplicate, skip it
        if m in dups:
            continue
        # now we need to test every other cycle and see if it is identical to this
        for n in range(m+1, len(cl)): # all but me!
            if n in dups: # donʻt bother
                continue
            if test_if_cycle_identical(cl[m], cl[n]):
                # we have a duplicate. we need to remove n
                dups.add(n)
    
    # now we can remove the duplicates
    for dup in sorted(dups, reverse=True):
        cl.pop(dup)

    # now we sort by length and then alpha
    cl = sorted(cl, key=lambda x: (len(x), x[0]), reverse=True)

    return cl


# some_cycles = [['0101', '1010'], ['1010', '0101'], ['1000'], ['1000']]
# print(remove_duplicates(some_cycles))
# removed duplicates: [['0101', '1010'], ['1000']]

def do_assignment():
    cycles_list = []
    answer_template = "In Fig. {figure} a network with N = {N} and K = {K} is fully defined. The time evolution of the 2^{N} = {total_states} states Σt is given for synchronous updating. One can observe {cycles_report} and {fixpoints_report}."

    for i in range(2**N):
        i_bin = np.binary_repr(i, width=N)
        starting_state = np.array([int(x) for x in i_bin])
        state = starting_state
        path = [starting_state]
        while True:
            new_state = update(state)
            idx_if_present = get_idx_of_np_array_in_list(new_state, path)
            if idx_if_present > -1:
                cycle = path[idx_if_present:] 
                cycles_list.append(cycle_to_string(cycle))
                break
            state = new_state
            path.append(new_state)

    # print(cycles_list)  # (before deduplication)
    cycles_list = remove_duplicates_and_sort(cycles_list)   # this is our very nice list of cycles
    # print(cycles_list)  # (after deduplication)

    # we need a count of cycles for each cycle length
    counts = OrderedDict()   # eg [(3, 1), (2, 3), (1, 2)] # (size, count)
    for cycle in cycles_list: # ordered
        cycle_length = len(cycle)
        if cycle_length in counts.keys():
            counts[cycle_length] += 1
        else:
            counts[cycle_length] = 1

    # print(counts)  SO MUCH DEBUGGING
    # eg, OrderedDict([(2, 1), (1, 2)])

    # for reporting.... lots of text manipulation
    cycles_report = ""
    fixpoints_report = ""

    for cycle_size in counts.keys():
        if cycle_size > 1:
            ess = 's' if counts[cycle_size] > 1 else '' # my favorite old trick
            cycles_report += f"{int_less_than_21_to_word(counts[cycle_size])} cycle{ess} of length {cycle_size},"
        else:
            fixpoints_report += f"{int_less_than_21_to_word(counts[cycle_size])} cycles of length 1 (fixpoints)"

    answer = answer_template.format(
        figure=figure,
        N=N,
        K=K,
        total_states=2**N,
        cycles_report=cycles_report,
        fixpoints_report=fixpoints_report
    )
    
    return answer

# format print so that, in the notebook, it wraps nicely
a = do_assignment()
wrapped_a = textwrap.fill(a, width=80)
print(wrapped_a)



In Fig. 1 a network with N = 4 and K = 2 is fully defined. The time evolution of
the 2^4 = 16 states Σt is given for synchronous updating. One can observe one
cycle of length 2, and two cycles of length 1 (fixpoints).


### References

Gros, C. (2008). Complex and Adaptive Dynamical Systems: A Primer. Springer Berlin Heidelberg. https://doi.org/10.1007/978-3-540-71874-1
